In [1]:
import os
import cv2
import glob
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

from sklearn.manifold import TSNE
import plotly.express as px

import tensorflow as tf
from tensorflow.keras.layers import Flatten
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

2023-05-30 16:05:37.297633: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data Utility

In [ ]:
root_path = "../FLIR_Data/"

# Assumes that all images are of the same shape...
def readRawData(source_base_path, verbose = False):
    """
    Reads raw image data O(n)
    
    Parameters
    -----------------
    source_base_path: String path of the root folder for the dataset 
    
    Returns
    -----------------
    X_data: returns an Array of Heterogenous (RGB) values of input data for the tensor network
    """

    image_name_arr = glob.glob(os.path.join(source_base_path, "*.png")) + glob.glob(os.path.join(source_base_path, "*.jpg"));
    image_name_arr_sorted = sorted(image_name_arr, key = lambda x:x[0:]);
    
    original = [];
    
    for infile in image_name_arr_sorted:
        original_img = cv2.imread(infile);
        original.append(original_img);
        
        if (verbose): # If set to true, the runtime is slower...
            plt.imshow(original_img);
            plt.show()
    
    return np.array(original);

X_data = read_data(root_path);

# Sanity Check
print("X_Training data shape:", X_data.shape);


In [ ]:
# FUTURE USE (Once we get morphological data...)

# def read_data(source_base_path, img_folder_name, seg_folder_name, verbose = False):
#     """
#     Reads the data and splits it from X_data and Labels O(n)
    
#     Parameters
#     -----------------
#     source_base_path: String path of the root folder for the dataset 
#     (Assumes that there is a seg_folder and img_folder.)
#     img_folder_name: String of the image folder name found within the source_base_path
#     seg_folder_name: String of the segmentation folder name found within the source_base_path
    
#     Returns
#     -----------------
#     X_data: returns an Array of Heterogenous (RGB) values of input data for the tensor network
#     Y_Data: returns an Array of Heterogenous (RGB) value of segmentation masks for the tensor network
#     max_dim: greatest height and width, respectively, in given images
#     """
    
#     img_base_path = source_base_path + "/" + img_folder_name + "/";
    
#     image_name_arr = glob.glob(os.path.join(img_base_path, "*.png")) + glob.glob(os.path.join(img_base_path, "*.tif"));
#     image_name_arr_sorted = sorted(image_name_arr, key = lambda x:x[0:]);
    
#     original = [];
#     masks = [];

#     max_height = 0
#     max_width = 0   
    
#     for infile in image_name_arr_sorted:
#         seg_path = infile.replace(img_folder_name, seg_folder_name); 
        
#         original_img = cv2.imread(infile);
#         seg_img = cv2.imread(seg_path);
        
#         height, width, _  = np.shape(original_img)
#         # Check if the height and width is greater than greatest
#         if height > max_height:
#             max_height = height

#         if width > max_width:
#             max_width = width

#         original.append(original_img);
#         masks.append(seg_img);
        
#         if (verbose): # If set to true, the runtime is slower...
#             plt.imshow(original_img);
#             plt.show()
#             plt.imshow(seg_img);
#             plt.show()
    
#     return original, masks, (max_height, max_width);

# X_data, Y_data, max_dim = read_data(root_path, "img", "seg");

# # Sanity Check
# print("X_Training data length:", len(X_data))
# print("Y_Training data length:", len(Y_data))
# print("Max_Dimensions:", max_dim)

# Feature Vector Embedding: Extraction of Features using ResNet 50-101

In [ ]:
model_101 = ResNetEncoder101()
model_50 = ResNetEncoder50()
features_vectors_101 = model_101.predict(Overall_Images, batch_size = 32)
features_vectors_50 = model_50.predict(Overall_Images, batch_size = 32)

print(features_vectors_101)
print(features_vectors_50)

## t-SNE Conversion and Visualization

In [ ]:
# scale and move the coordinates so they fit [0; 1] range
def scale_to_01_range(x):
    value_range = (np.max(x) - np.min(x))
    starts_from_zero = x - np.min(x)
    return starts_from_zero / value_range

# ======================================================

# We want to get TSNE embedding with 2 dimensions
n_components = 2
tsne_101 = TSNE(n_components)
tsne_X_101 = tsne_101.fit_transform(features_vectors_101)
tsne_X_101.shape

tx_101 = tsne_X_101[:, 0]
ty_101 = tsne_X_101[:, 1]
 
tx_101 = scale_to_01_range(tx_101)
ty_101 = scale_to_01_range(ty_101)

# ======================================================

# We want to get TSNE embedding with 2 dimensions
n_components = 2
tsne_50 = TSNE(n_components)
tsne_X_50 = tsne_50.fit_transform(features_vectors_50)
tsne_X_50.shape

tx_50 = tsne_X_50[:, 0]
ty_50 = tsne_X_50[:, 1]
 
tx_50 = scale_to_01_range(tx_50)
ty_50 = scale_to_01_range(ty_50)

In [ ]:
fig = px.scatter(x = tx_101, y = ty_101, color=Overall_Labels.flatten())
fig.update_layout(
    title="t-SNE visualization of Custom Dataset with RESNET-101",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig.show()

fig = px.scatter(x = tx_50, y = ty_50, color=Overall_Labels.flatten())
fig.update_layout(
    title="t-SNE visualization of Custom Dataset with RESNET-50",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig.show();